In [1]:
running = False
try:
    sock.close()
except:
    pass

In [2]:
from parser import public_construct, public_match

In [3]:
from utils import SocketReader

In [4]:
import math

In [5]:
rmeote_offer = """v=0
o=- 2383922543316812562 2 IN IP4 127.0.0.1
s=-
t=0 0
a=group:BUNDLE 0
a=extmap-allow-mixed
a=msid-semantic: WMS
m=application 35423 UDP/DTLS/SCTP webrtc-datachannel
c=IN IP4 35.226.7.11
a=candidate:4207328549 1 udp 2113937151 aa634479-e1c4-42b4-abc7-fda55eca7536.local 35423 typ host generation 0 network-cost 999
a=candidate:4285477453 1 udp 1677729535 35.226.7.11 35423 typ srflx raddr 0.0.0.0 rport 0 generation 0 network-cost 999
a=ice-ufrag:ctXb
a=ice-pwd:P85I1LFzAEQeBIdOB25YHQTZ
a=ice-options:trickle
a=fingerprint:sha-256 D6:32:08:60:79:B5:C9:08:28:97:24:2E:9D:2E:0A:A8:0B:C3:42:37:E4:E9:3D:CD:FF:7D:B4:51:66:BB:B1:09
a=setup:actpass
a=mid:0
a=sctp-port:5000
a=max-message-size:262144

"""

In [6]:
for line in rmeote_offer.splitlines():
    if 'a=candidate:' in line and '.local' not in line:
        res = line.split(' ')
        target_ip = res[4]
        target_port = int(res[5])
    if 'a=ice-ufrag:' in line:
        remote_ufrag = line.split(':')[1]
    if 'a=ice-pwd:' in line:
        remote_pwd = line.split(':')[1]

target_ip_num = 0
for i, seg in enumerate(target_ip.split('.')):
    target_ip_num += int(seg) * (256**(3-i))

target_ip, target_port, remote_ufrag, remote_pwd

('35.226.7.11', 35423, 'ctXb', 'P85I1LFzAEQeBIdOB25YHQTZ')

In [7]:
import random, string
len_remote_ufrag = len(remote_ufrag)
# len_remote_ufrag = 4 # chrome
local_ufrag = ''.join(random.choice(string.ascii_uppercase + string.ascii_lowercase + string.digits) for _ in range(16 - len_remote_ufrag - 1))
local_pwd = 'AgLw44VDUK531eYVCMpZMAjF'
local_ufrag

'CXwe8jd6UHc'

In [8]:
from stun import do_stun
import random
sport = random.randrange(30000, 50000)
ip, port = do_stun(sport)
f"found {ip}:{port} while source port is {sport}"

'found 143.178.72.250:47222 while source port is 47222'

In [9]:
from cryptography.hazmat.primitives.asymmetric import ec
from cryptography.hazmat.primitives import serialization
import OpenSSL.crypto

own_cert_private_key = ec.generate_private_key(ec.SECP256R1())

own_cert_private_key_bytes = own_cert_private_key.private_bytes(
    encoding=serialization.Encoding.PEM,
    format=serialization.PrivateFormat.PKCS8,
    encryption_algorithm=serialization.NoEncryption()
)


privkey = OpenSSL.crypto.load_privatekey(OpenSSL.crypto.FILETYPE_PEM, own_cert_private_key_bytes)

pubkey = OpenSSL.crypto.load_publickey(OpenSSL.crypto.FILETYPE_PEM,
                                     own_cert_private_key.public_key().public_bytes(
                                         encoding=serialization.Encoding.PEM,
                                         format=serialization.PublicFormat.SubjectPublicKeyInfo,
                                     )
                                    )

local_cert = OpenSSL.crypto.X509()
local_cert.set_version(2)
local_cert.gmtime_adj_notBefore(0)
local_cert.gmtime_adj_notAfter(1 * 24 * 60 * 60)
local_cert.get_issuer().CN = 'WebRTC'
local_cert.get_subject().CN = 'WebRTC'
local_cert.set_serial_number(random.randrange(1<<63, 1<<64))
local_cert.set_pubkey(pubkey)
local_cert.sign(privkey, 'sha256') 
print(OpenSSL.crypto.dump_certificate(OpenSSL.crypto.FILETYPE_TEXT, local_cert).decode('utf-8')) 
local_cert_sha = local_cert.digest('sha256').decode('utf-8')
local_cert_sha

Certificate:
    Data:
        Version: 3 (0x2)
        Serial Number:
            91:e8:4a:38:e8:ad:26:b7
        Signature Algorithm: ecdsa-with-SHA256
        Issuer: CN=WebRTC
        Validity
            Not Before: Jul  4 22:21:19 2024 GMT
            Not After : Jul  5 22:21:19 2024 GMT
        Subject: CN=WebRTC
        Subject Public Key Info:
            Public Key Algorithm: id-ecPublicKey
                Public-Key: (256 bit)
                pub:
                    04:28:d2:cb:12:8f:ad:bc:d3:11:65:97:4d:eb:c0:
                    24:94:c4:f0:3c:2e:fe:dd:82:60:a6:e5:61:6f:b6:
                    14:1b:24:36:a9:42:01:02:26:57:3f:8d:17:3b:0a:
                    e1:ec:46:8d:be:ee:45:1c:f5:22:11:9c:95:d1:d4:
                    0e:0e:b6:8e:0f
                ASN1 OID: prime256v1
                NIST CURVE: P-256
    Signature Algorithm: ecdsa-with-SHA256
    Signature Value:
        30:46:02:21:00:88:5f:77:f2:24:37:f2:bd:9d:2b:f6:05:72:
        55:36:c0:8f:32:ae:4e:5d:92:85:bc

'39:90:97:D9:BF:B3:7D:43:AF:A5:AF:BA:AD:99:ED:EF:DF:87:88:F6:33:0B:84:58:2F:CC:AE:67:2E:6A:A3:CC'

In [10]:
local_offer = f"""v=0
o=- 7407514470269221864 2 IN IP4 127.0.0.1
s=-
t=0 0
a=group:BUNDLE 0
a=extmap-allow-mixed
a=msid-semantic: WMS
m=application {port} UDP/DTLS/SCTP webrtc-datachannel
c=IN IP4 {ip}
a=candidate:3055715734 1 udp 1677729535 {ip} {port} typ srflx raddr 0.0.0.0 rport 0 generation 0 network-cost 999
a=ice-ufrag:{local_ufrag}
a=ice-pwd:{local_pwd}
a=ice-options:trickle
a=fingerprint:sha-256 {local_cert_sha}
a=setup:active
a=mid:0
a=sctp-port:5000
a=max-message-size:262144
"""
print(local_offer)

v=0
o=- 7407514470269221864 2 IN IP4 127.0.0.1
s=-
t=0 0
a=group:BUNDLE 0
a=extmap-allow-mixed
a=msid-semantic: WMS
m=application 47222 UDP/DTLS/SCTP webrtc-datachannel
c=IN IP4 143.178.72.250
a=candidate:3055715734 1 udp 1677729535 143.178.72.250 47222 typ srflx raddr 0.0.0.0 rport 0 generation 0 network-cost 999
a=ice-ufrag:CXwe8jd6UHc
a=ice-pwd:AgLw44VDUK531eYVCMpZMAjF
a=ice-options:trickle
a=fingerprint:sha-256 39:90:97:D9:BF:B3:7D:43:AF:A5:AF:BA:AD:99:ED:EF:DF:87:88:F6:33:0B:84:58:2F:CC:AE:67:2E:6A:A3:CC
a=setup:active
a=mid:0
a=sctp-port:5000
a=max-message-size:262144



In [11]:
stun_header_spec = """
2@zeroes
-14@message_type
-16@message_length
-32@magic_cookie
-96@transaction_id
"""

padding_0_attribute_spec = """
...(
    16==0x20?16@xor_mapped_ip(
        8@zeroes
        -8@family
        -16@port
        -32@ip
    )
    :16==0x06?16@username()
    :16==0x08?16@message_integrity(20B@hmac)
    :16==0x25?16@use_candidate()
    :16==0x8029?16@ice_controlled(8B@tie_breaker)
    :16==0x802a?16@ice_controlling(8B@tie_breaker)
    :16==0x24?16@priority(4B@priority)
    :16==0x8028?16@fingerprint(4B@crc32)
    :16@selector-16@data()
)
"""

import random
magic_cookie = 0x2112A442 # constant

In [12]:
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.hmac import HMAC

def get_message_integrity(msg, key):
    msg = bytearray(msg)
    msg[3] = len(msg) + 24 - 20
    hm = HMAC(key, hashes.SHA1())
    hm.update(msg)
    return hm.finalize()

def append_message_integrity(msg, key):
    hmac = get_message_integrity(msg, key)
    attrs = [{"message_integrity": {"hmac": hmac}, "_selector": 0x08}]
    msg = bytearray(msg)
    msg += public_construct(padding_0_attribute_spec, attrs)
    msg[3] = len(msg) - 20
    return msg

import binascii
def append_fingerprint(msg):
    msg = bytearray(msg)
    msg[3] = len(msg) + 8 - 20
    crc32 = binascii.crc32(msg) ^ 0x5354554e
    attrs = [{"fingerprint": {"crc32": crc32}, "_selector": 0x8028}]
    msg += public_construct(padding_0_attribute_spec, attrs)
    return msg
    


# v = """000100502112a4427139396c466f2b2f642f48650006000952556d333a4e6b4b36000000c0570004000003e7802a00084f02cc537c6985ff00250000002400046e001eff00080014ed4243520504f6eeefe90146a25142054ab03a4380280004f0c40a27"""
# msgm = bytes.fromhex(v)
# should_be = msgm[-28:-8].hex()
# print('should be', should_be)
# msgm = bytearray(msgm[:-8][:-24])
# key = b'7bjW8vqNNZPvdFDCa3IJ3J7E'
# print(v)
# print(append_fingerprint(append_message_integrity(msgm, key)).hex())


In [13]:
def create_response(transaction_id):
    attributes = [
        {'xor_mapped_ip': {'zeroes': 0x00,
       'family': 0x01,
       'port': target_port  ^ (magic_cookie >> 16),
       'ip': target_ip_num  ^ (magic_cookie)},
      '_selector': 0x0020}
    ]
    
    msg = public_construct(padding_0_attribute_spec, attributes)
    
    stun_header = {
        "zeroes": 0,
        # +--+--+-+-+-+-+-+-+-+-+-+-+-+-+
        # |M |M |M|M|M|C|M|M|M|C|M|M|M|M|
        # |11|10|9|8|7|1|6|5|4|0|3|2|1|0|
        # +--+--+-+-+-+-+-+-+-+-+-+-+-+-+
        "message_type": 0b00000100000001,
        "message_length": len(msg)+24,
        "magic_cookie": magic_cookie,
        "transaction_id": transaction_id,
    }
    
    msg = public_construct(stun_header_spec, stun_header) + msg
    msg = append_message_integrity(msg, local_pwd.encode('utf-8'))
    msg = append_fingerprint(msg)
    return msg

def create_request():
    attributes = [
        {"username": remote_ufrag+':'+local_ufrag, "_selector": 0x06},
        # {"use_candidate": "", "_selector": 0x25},
        # {"ice_controlling": {"tie_breaker": 0x6569bb58ab364c66}, "_selector": 0x802a},
        {"ice_controlled": {"tie_breaker": 0x6569bb58ab364c66}, "_selector": 0x8029},
        {"priority": {"priority": 0x6e001eff}, "_selector": 0x24},
    ]
    
    msg = public_construct(padding_0_attribute_spec, attributes)
    
    stun_header = {
        "zeroes": 0,
        # +--+--+-+-+-+-+-+-+-+-+-+-+-+-+
        # |M |M |M|M|M|C|M|M|M|C|M|M|M|M|
        # |11|10|9|8|7|1|6|5|4|0|3|2|1|0|
        # +--+--+-+-+-+-+-+-+-+-+-+-+-+-+
        "message_type": 0b00000000000001,
        "message_length": len(msg)+24,
        "magic_cookie": magic_cookie,
        "transaction_id": random.randrange(1<<96),
    }
    
    msg = public_construct(stun_header_spec, stun_header) + msg
    msg = append_message_integrity(msg, remote_pwd.encode('utf-8'))
    msg = append_fingerprint(msg)
    return msg


In [14]:
try:
    sock.close()
except:
    pass

sock = SocketReader(target_ip, target_port, 'udp')

sock.connect(source_port=sport)

In [15]:
running = True
import time
from queue import Queue

dts_queue = Queue(maxsize=10)

last_received = time.time()

def handler():
    while running:
        resp = sock.read_once()
        if resp[1] == 0xfe and resp[2] == 0xfd:
            dts_queue.put(resp)
        res, rem = public_match(stun_header_spec, resp)
        if res['message_type'].int() != 0b00000000000001:
            continue
        pkg = create_response(res['transaction_id'])
        sock.send(pkg)
        print('+', end='')
        last_received = time.time()

def sender():
    while running:
        sock.send(create_request())
        print('*', end='')
        time.sleep(2 if time.time() - last_received < 6 else 0.2)

import threading
threading.Thread(target=handler).start()
threading.Thread(target=sender).start()

*+++++**+****+*************+*************+*************+**************+*************+*************+*************+**************+*************+*************+*************+*************+**************+*************+*************+*************+*************+**************+*************+*************+*************+*************+**************+*************+*************+*************+**************+*************+*************+*************+*************+**************+*************+*************+*************+*************+**************+*************+*************+*************+*************+**************+*************+*************+*************+*************+**************+*************+*************+*************+**************+*************+*************+*************+*************+**************+*************+*************+*************+*************+**************+*************+*************+*************+*************+**************+*************+*************+*************+******

Exception in thread Thread-6 (sender):
Traceback (most recent call last):
  File "/usr/lib/python3.12/threading.py", line 1073, in _bootstrap_inner
    self.run()
  File "/home/mie/.virtualenvs/notebooks/lib/python3.12/site-packages/ipykernel/ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "/usr/lib/python3.12/threading.py", line 1010, in run
    self._target(*self._args, **self._kwargs)
  File "/tmp/ipykernel_3187185/4249774954.py", line 24, in sender
  File "/home/mie/notebooks/utils.py", line 17, in send
    self.sock.sendall(data)
OSError: [Errno 101] Network is unreachable


In [16]:
# running = False
# sock.close()


In [17]:
transcript = b''

In [18]:
handshake_spec = """
8==1?24@client_hello(
    16@version
    -32B@random
    -8@session_id()
    -8@cookie()
    -16@suites(...16)
    -8@compression()
    -16@extensions(
        ...(
            16==23?16@extended_master_secret()
            :16==0xff01?16@renegotiation_info(8@renegotiation_info())
            :16==10?16@ext_supported_curves(
                16@data(...16)
            )
            :16==11?16@ext_supported_points(
                8@supported_points()
            )
            :16==35?16@ext_session_tickets()
            :16==13?16@ext_signature_algorithms(
                16@data(...16)
            )
            :16==14?16@use_srtp(
                16@data(...16)
                -8@mki()
            )
            :16@selector-16@raw_extension()
        )
    )
)
:8==2?24@server_hello(
    16@version
    -32B@random
    -8@session_id()
    -16@suite
    -8@compression
    -16@extensions(
        ...(
            16==23?16@ext_master_secret()
            :16==11?16@ext_supported_points(
                8@supported_points()
            )
            :16==35?16@ext_session_tickets()
            :16==5?16@ext_status_request(
                8@status_type
                -16@responder_id_list
                -16@request_extensions
            )
            :16==0xff01?16@ext_renegotiation_info(
                8@secure_renegotitation()
            )
            :16==14?16@use_srtp(
                16@data(...16)
                -8@mki()
            )
            :16@selector-16@raw_extension()
        )
    )
)
:8==11?24@certificates_data(
    24@certificates(
        ...(
            24@certificate()
        )
    )
)
:8==12?24@server_key_exchange(
    8@curve_type
    -16@named_curve
    -8@pubkey()
    -16@sig_algorithm
    -16@signature()
)
:8==13?24@certificate_request(
    8@cert_types(...(8@cert_type))
    -16@sig_hash_algs(...(16@sig_hash_alg))
    -16@distinguished_names()
)
:8==14?24@server_hello_done()
:8==16?24@client_key_exchange(
    8@pubkey()
)
:8==15?24@certificate_verify(
    16@sig_algorithm
    -16@signature()
)
:8==4?24@new_session_ticket(
    4B@lifetime_hint
    -16@session_ticket()
)
:8==20?24@finished(12B@checksum)
"""

fragment_spec = """
4B@type_and_length
-16@message_sequence
-24@fragment_offset
-24@fragment()
"""

dtls_record_spec = """
8==22?16@tls_version-16@epoch-6B@sequence_number-16@tls_handshake(%s)
:8==20?16@tls_version-16@epoch-6B@sequence_number-16@change_cipher_spec_message()
"""%fragment_spec

In [19]:
client_hello = {
    'client_hello': {
        'version': 0xfefd,
        'random': bytes.fromhex('2dd93a4dd171b16cef3fd22029ab20c59938934d74b4c597b762f16ff261735b'),
        'session_id': '',
        'cookie': '',
        # 'suites': [0xc02b],
        'suites': [0xc02b, 0xc02f, 0xcca9, 0xcca8, 0xc009, 0xc013, 0xc00a, 0xc014, 0x009c, 0x002f, 0x0035],
        'compression': 0x00,
        'extensions': [
            {
                'extended_master_secret': '', '_selector': 23
            },
            {
                'renegotiation_info': {
                    'renegotiation_info': '',
                }, '_selector': 0xff01
            },
            {
                'ext_supported_curves': {
                    # 'data': [0x001d]
                    'data': [0x001d, 0x0017, 0x0018]
                }, '_selector': 0x000a
            },
            {
                'ext_supported_points': {
                    'supported_points': 0x00
                }, '_selector': 0x000b
            },
            {
                'ext_session_tickets': '', '_selector': 35
            },
            {
                'ext_signature_algorithms': {
                    # 'data': [0x0403]
                    'data': [0x0403, 0x0804, 0x0401, 0x0503, 0x0805, 0x0501, 0x0806, 0x0601, 0x0201]
                }, '_selector': 0x000d
            },
            {
                'use_srtp': {
                    # 'data': [0x0001],
                    'data': [0x0001, 0x0008, 0x0007],
                    'mki': ''
                }, '_selector': 14
            }
        ]
    },
    '_selector': 0x01
}

client_hello_encoded = public_construct(handshake_spec, client_hello)



dtls_record = {
    "tls_version": 0xfefd,
    "tls_handshake": {
        'type_and_length': client_hello_encoded[:4],
        'message_sequence': 0,
        'fragment_offset': 0,
        'fragment': client_hello_encoded[4:],
    },
    "epoch": 0,
    "sequence_number": 0,
    "_selector": 22
}
dtls_record_encoded = public_construct(dtls_record_spec, dtls_record)


In [20]:
transcript += dtls_record_encoded[13:]
sock.send(dtls_record_encoded)

In [21]:
# msg = bytes.fromhex("16fefd00000000000000000050020000440000000000000044fefd666d5aac48243def6a5466b05ba62cbf15f4eea136c99df59f782732ef9d558900c02b00001c00170000ff01000100000b0002010000230000000e0005000200010016fefd0000000000000001012c0b000120000100000000012000011d00011a308201163081bda0030201020209008bff228a3f27c618300a06082a8648ce3d0403023011310f300d06035504030c06576562525443301e170d3234303631343039303932315a170d3234303731353039303932315a3011310f300d06035504030c065765625254433059301306072a8648ce3d020106082a8648ce3d03010703420004a7990ef7c6e5f1a8c29e5cdb4b3e4cb1dc376b48ffd392751dc6fd763de583ee77a65da77341fbeed4e83fb1a09d4f1c621c89d9d974e50a0339ed4d380c46bb300a06082a8648ce3d0403020348003045022100a07ff3282ccb96a42387a8851d0b26a42a5f1082789fe8f682df50fbccb35a8202201dc64921ff9358f5bdec2a644c0597312f5cea62f295b875daf0e6bb1fe3c3e916fefd0000000000000002007c0c000070000200000000007003001d20eb7088c2a60f227ef9bdb01a5082a34048ff182b9e142afcb40c07b368fea016040300483046022100fec9df11479090a9553f1497c7707e4146100af1f9e5f36c60f9a4c421c5f419022100e494e6bb2765926893edd08f028cf692d69e7534affa89756802768a7bdc956816fefd000000000000000300250d00001900030000000000190201400012040308040401050308050501080606010201000016fefd0000000000000004000c0e0000000004000000000000")
msg = dts_queue.get()

In [22]:
packet, rem = public_match(dtls_record_spec, msg)
transcript += msg[13:-len(rem)]
packet

{'tls_version': 0xfefd,
 '_selector': 0x16,
 'epoch': 0x0000,
 'sequence_number': 0x000000000000,
 'tls_handshake': {'type_and_length': 0x02000044,
  'message_sequence': 0x0000,
  'fragment_offset': 0x000000,
  'fragment': 0xfefd6687207c3e922ce5859ab6a3d1ed3ca33eade004b00d67c6f04773c6eb7eac7600c02b00001c00170000ff01000100000b0002010000230000000e00050002000100}}

In [23]:
record = packet['tls_handshake']['type_and_length'].bytes() + packet['tls_handshake']['fragment'].bytes()
server_hello, _ = public_match(handshake_spec, record)
server_hello

{'server_hello': {'version': 0xfefd,
  'random': 0x6687207c3e922ce5859ab6a3d1ed3ca33eade004b00d67c6f04773c6eb7eac76,
  'session_id': '',
  'suite': 0xc02b,
  'compression': 0x00,
  'extensions': [{'ext_master_secret': '', '_selector': 0x0017},
   {'ext_renegotiation_info': {'secure_renegotitation': ''},
    '_selector': 0xff01},
   {'ext_supported_points': {'supported_points': 0x00}, '_selector': 0x000b},
   {'ext_session_tickets': '', '_selector': 0x0023},
   {'use_srtp': {'data': [0x0001], 'mki': ''}, '_selector': 0x000e}]},
 '_selector': 0x02}

In [24]:
# rem = bytes.fromhex('16fefd0000000000000001012c0b000120000100000000012000011d00011a308201163081bda003020102020900b388ba8f0397e380300a06082a8648ce3d0403023011310f300d06035504030c06576562525443301e170d3234303631343039303931365a170d3234303731353039303931365a3011310f300d06035504030c065765625254433059301306072a8648ce3d020106082a8648ce3d030107034200045d4f0a2e4e3e0bfe83012c62d4649a43fa9bb46aa4579fa59b739e6463a0a8c0395545195f81f8895480128cda85a9c79699baca8b413065c7ef013c226be5e6300a06082a8648ce3d040302034800304502206e8136052a4d8689988d3f1205882d1d224cd9b2543c1e9463ac9bae5aee87f90221009c4d8b5e119541c461c4d704121abb097ee983dddb0c7ea4c8fd06ba08623cea')
packet, rem1 = public_match(dtls_record_spec, rem)
transcript += rem[13:-len(rem1)]
packet

{'tls_version': 0xfefd,
 '_selector': 0x16,
 'epoch': 0x0000,
 'sequence_number': 0x000000000001,
 'tls_handshake': {'type_and_length': 0x0b00011e,
  'message_sequence': 0x0001,
  'fragment_offset': 0x000000,
  'fragment': 0x00011b000118308201143081bca003020102020802dfcaa219fd660c300a06082a8648ce3d0403023011310f300d06035504030c06576562525443301e170d3234303730333232323034395a170d3234303830333232323034395a3011310f300d06035504030c065765625254433059301306072a8648ce3d020106082a8648ce3d03010703420004e4998b6a7fd4e7c4ad7fb4f322fb2ed2350b544f4127f71c05a17b44693b29b99136a0ac7dbd332cc5fab7b779dd45f37587abdeabb560edbccfdcd1fb56a524300a06082a8648ce3d040302034700304402207db4107ef9ecaf13f4d944500be1977268a3db646e77deeccb7295692b6610e902204964d9856bbb5d3254a6c746c00b7fe4ad22233c46f5ce9e0953b2e1cd2aae36}}

In [25]:
record = packet['tls_handshake']['type_and_length'].bytes() + packet['tls_handshake']['fragment'].bytes()
peer_cert, _ = public_match(handshake_spec, record)
peer_cert

{'certificates_data': {'certificates': [{'certificate': 0x308201143081bca003020102020802dfcaa219fd660c300a06082a8648ce3d0403023011310f300d06035504030c06576562525443301e170d3234303730333232323034395a170d3234303830333232323034395a3011310f300d06035504030c065765625254433059301306072a8648ce3d020106082a8648ce3d03010703420004e4998b6a7fd4e7c4ad7fb4f322fb2ed2350b544f4127f71c05a17b44693b29b99136a0ac7dbd332cc5fab7b779dd45f37587abdeabb560edbccfdcd1fb56a524300a06082a8648ce3d040302034700304402207db4107ef9ecaf13f4d944500be1977268a3db646e77deeccb7295692b6610e902204964d9856bbb5d3254a6c746c00b7fe4ad22233c46f5ce9e0953b2e1cd2aae36}]},
 '_selector': "
 "}

In [26]:

first_server_cert = None
for cert in peer_cert['certificates_data']['certificates']:
    x509 = OpenSSL.crypto.load_certificate(OpenSSL.crypto.FILETYPE_ASN1, cert['certificate'].bytes())
    if first_server_cert is None:
        first_server_cert = x509
    print(OpenSSL.crypto.dump_certificate(OpenSSL.crypto.FILETYPE_TEXT, x509).decode('utf-8')) 

Certificate:
    Data:
        Version: 3 (0x2)
        Serial Number: 207106905451881996 (0x2dfcaa219fd660c)
        Signature Algorithm: ecdsa-with-SHA256
        Issuer: CN=WebRTC
        Validity
            Not Before: Jul  3 22:20:49 2024 GMT
            Not After : Aug  3 22:20:49 2024 GMT
        Subject: CN=WebRTC
        Subject Public Key Info:
            Public Key Algorithm: id-ecPublicKey
                Public-Key: (256 bit)
                pub:
                    04:e4:99:8b:6a:7f:d4:e7:c4:ad:7f:b4:f3:22:fb:
                    2e:d2:35:0b:54:4f:41:27:f7:1c:05:a1:7b:44:69:
                    3b:29:b9:91:36:a0:ac:7d:bd:33:2c:c5:fa:b7:b7:
                    79:dd:45:f3:75:87:ab:de:ab:b5:60:ed:bc:cf:dc:
                    d1:fb:56:a5:24
                ASN1 OID: prime256v1
                NIST CURVE: P-256
    Signature Algorithm: ecdsa-with-SHA256
    Signature Value:
        30:44:02:20:7d:b4:10:7e:f9:ec:af:13:f4:d9:44:50:0b:e1:
        97:72:68:a3:db:64:6e:77:de:ec

In [27]:
first_server_cert.digest('sha256')
# TODO: assert digest

b'D6:32:08:60:79:B5:C9:08:28:97:24:2E:9D:2E:0A:A8:0B:C3:42:37:E4:E9:3D:CD:FF:7D:B4:51:66:BB:B1:09'

In [28]:
packet, rem2 = public_match(dtls_record_spec, rem1)
transcript += rem1[13:-len(rem2)]
packet

{'tls_version': 0xfefd,
 '_selector': 0x16,
 'epoch': 0x0000,
 'sequence_number': 0x000000000002,
 'tls_handshake': {'type_and_length': 0x0c00006e,
  'message_sequence': 0x0002,
  'fragment_offset': 0x000000,
  'fragment': 0x03001d20046ab0c37da94ce41f4e1069f28173e4030302dabca5593600fa7f532cc5f85d040300463044022029af60279f744f7b3ee3a1699cc9faa692e27c1d3637c2e8d36fad9efe08548102200a3d2c85f61d0c214b11156fdd933b6f5c9a8df7b5af991dd9eae4a9c587d72b}}

In [29]:
record = packet['tls_handshake']['type_and_length'].bytes() + packet['tls_handshake']['fragment'].bytes()
server_key, _ = public_match(handshake_spec, record)
server_key

{'server_key_exchange': {'curve_type': 0x03,
  'named_curve': 0x001d,
  'pubkey': 0x046ab0c37da94ce41f4e1069f28173e4030302dabca5593600fa7f532cc5f85d,
  'sig_algorithm': 0x0403,
  'signature': 0x3044022029af60279f744f7b3ee3a1699cc9faa692e27c1d3637c2e8d36fad9efe08548102200a3d2c85f61d0c214b11156fdd933b6f5c9a8df7b5af991dd9eae4a9c587d72b},
 '_selector': "
 "}

In [30]:
server_key_sign_spec = """32B@client_random-32B@server_random-8@curve_type-16@named_curve-8@pubkey()"""
server_key_sign = {
    "client_random": client_hello['client_hello']['random'],
    "server_random": server_hello['server_hello']['random'],
    "curve_type": server_key['server_key_exchange']['curve_type'],
    "named_curve": server_key['server_key_exchange']['named_curve'],
    "pubkey": server_key['server_key_exchange']['pubkey'],
}

enc = public_construct(server_key_sign_spec, server_key_sign)

first_server_cert.get_pubkey().to_cryptography_key().verify(server_key['server_key_exchange']['signature'].bytes(), enc, ec.ECDSA(hashes.SHA256()))


In [31]:
packet, rem3 = public_match(dtls_record_spec, rem2)
transcript += rem2[13:-len(rem3)]
packet

{'tls_version': 0xfefd,
 '_selector': 0x16,
 'epoch': 0x0000,
 'sequence_number': 0x000000000003,
 'tls_handshake': {'type_and_length': 0x0d000019,
  'message_sequence': 0x0003,
  'fragment_offset': 0x000000,
  'fragment': 0x02014000120403080404010503080505010806060102010000}}

In [32]:
record = packet['tls_handshake']['type_and_length'].bytes() + packet['tls_handshake']['fragment'].bytes()
certificate_request, _ = public_match(handshake_spec, record)
certificate_request

{'certificate_request': {'cert_types': [{'cert_type': 0x01},
   {'cert_type': "@"}],
  'sig_hash_algs': [{'sig_hash_alg': 0x0403},
   {'sig_hash_alg': 0x0804},
   {'sig_hash_alg': 0x0401},
   {'sig_hash_alg': 0x0503},
   {'sig_hash_alg': 0x0805},
   {'sig_hash_alg': 0x0501},
   {'sig_hash_alg': 0x0806},
   {'sig_hash_alg': 0x0601},
   {'sig_hash_alg': 0x0201}],
  'distinguished_names': ''},
 '_selector': "
 "}

In [33]:
packet, rem4 = public_match(dtls_record_spec, rem3)
transcript += rem3[13:]
packet

{'tls_version': 0xfefd,
 '_selector': 0x16,
 'epoch': 0x0000,
 'sequence_number': 0x000000000004,
 'tls_handshake': {'type_and_length': 0x0e000000,
  'message_sequence': 0x0004,
  'fragment_offset': 0x000000,
  'fragment': ''}}

In [34]:
record = packet['tls_handshake']['type_and_length'].bytes() + packet['tls_handshake']['fragment'].bytes()
server_hello_done, _ = public_match(handshake_spec, record)
server_hello_done

{'server_hello_done': '', '_selector': 0x0e}

In [35]:
dtls_message_encoded = b''

In [36]:
certificate = {
    'certificates_data':
        {
            'certificates':
                 [{'certificate': OpenSSL.crypto.dump_certificate(OpenSSL.crypto.FILETYPE_ASN1, local_cert)}]
        },
 '_selector': 11
}

certificate_encoded = public_construct(handshake_spec, certificate)



dtls_record = {
    "tls_version": 0xfefd,
    "tls_handshake": {
        'type_and_length': certificate_encoded[:4],
        'message_sequence': 1,
        'fragment_offset': 0,
        'fragment': certificate_encoded[4:],
    },
    "epoch": 0,
    "sequence_number": 1,
    "_selector": 22
}
dtls_record_encoded = public_construct(dtls_record_spec, dtls_record)
transcript += dtls_record_encoded[13:]
dtls_message_encoded += dtls_record_encoded

In [37]:
from cryptography.hazmat.primitives.asymmetric.x25519 import X25519PrivateKey, X25519PublicKey

client_private_key = X25519PrivateKey.generate()
client_public_key = client_private_key.public_key().public_bytes(serialization.Encoding.Raw, serialization.PublicFormat.Raw)


In [38]:

client_key_exchange = {
    'client_key_exchange': {
        'pubkey': client_public_key,
    },
 '_selector': 16
}

client_key_exchange_encoded = public_construct(handshake_spec, client_key_exchange)



dtls_record = {
    "tls_version": 0xfefd,
    "tls_handshake": {
        'type_and_length': client_key_exchange_encoded[:4],
        'message_sequence': 2,
        'fragment_offset': 0,
        'fragment': client_key_exchange_encoded[4:],
    },
    "epoch": 0,
    "sequence_number": 2,
    "_selector": 22
}
dtls_record_encoded = public_construct(dtls_record_spec, dtls_record)
transcript += dtls_record_encoded[13:]
dtls_message_encoded += dtls_record_encoded

In [39]:
signature = own_cert_private_key.sign(transcript, ec.ECDSA(hashes.SHA256()))

In [40]:

certificate_verify = {
    'certificate_verify': {
        'sig_algorithm': 0x0403,
        'signature': signature,
    },
 '_selector': 15
}

certificate_verify_encoded = public_construct(handshake_spec, certificate_verify)



dtls_record = {
    "tls_version": 0xfefd,
    "tls_handshake": {
        'type_and_length': certificate_verify_encoded[:4],
        'message_sequence': 3,
        'fragment_offset': 0,
        'fragment': certificate_verify_encoded[4:],
    },
    "epoch": 0,
    "sequence_number": 3,
    "_selector": 22
}
dtls_record_encoded = public_construct(dtls_record_spec, dtls_record)
# transcript += dtls_record_encoded[13:]
certificate_verify_message_encoded = dtls_record_encoded
dtls_message_encoded += dtls_record_encoded

In [41]:

dtls_record = {
    "tls_version": 0xfefd,
    "epoch": 0,
    "sequence_number": 4,
    "change_cipher_spec_message": 0x01,
    "_selector": 20
}
dtls_record_encoded = public_construct(dtls_record_spec, dtls_record)
dtls_message_encoded += dtls_record_encoded

In [42]:
from cryptography.hazmat.primitives import hashes, hmac

def HMAC_SHA256(secret, val):
    h = hmac.HMAC(secret, hashes.SHA256())
    h.update(val)
    return h.finalize()

def P_SHA256(secret, seed, l):
    b = bytearray()
    A = HMAC_SHA256(secret, seed)
    while len(b) < l:
        b += HMAC_SHA256(secret, A + seed)
        A = HMAC_SHA256(secret, A)
    return b[:l]

def PRF(secret, label, seed, l):
    return P_SHA256(secret, label + seed, l)

In [43]:
def calculate_sha256(data):
    digest = hashes.Hash(hashes.SHA256())
    digest.update(data)
    return digest.finalize()

In [44]:
shared_key = client_private_key.exchange(X25519PublicKey.from_public_bytes(server_key['server_key_exchange']['pubkey'].bytes()))
print("calculated shared key", shared_key)

calculated shared key b'\xed\xfd8\xbd\xe04\xc7t\xa2,\x97\xb2x\xba(\x17\xca+\x82\x94\xc3kr\xc1\n#\x16\xd0\xb6\x17\x1bL'


In [45]:
master_key = PRF(
    shared_key,
    b"extended master secret",
    # client_hello['client_hello']['random'].bytes() + server_hello['server_hello']['random'],
    calculate_sha256(transcript),
    48,
)
master_key.hex()

'36944df856ab5821ba72070069ea302d6899dcf0bdd3c330854f77a3b0513524fa9e2a112e0dd09124cd8f672ea55e60'

In [46]:
keyLen = 16
ivLen  = 4

raw_keys = PRF(
    master_key,
    b"key expansion",
    server_hello['server_hello']['random'].bytes() + client_hello['client_hello']['random'],
    40,
)

client_write_key = raw_keys[:keyLen]
raw_keys = raw_keys[keyLen:]

server_write_key = raw_keys[:keyLen]
raw_keys = raw_keys[keyLen:]

client_write_iv = raw_keys[:ivLen]
raw_keys = raw_keys[ivLen:]

server_write_iv = raw_keys[:ivLen]

In [47]:
transcript += certificate_verify_message_encoded[13:]

In [48]:
client_verify_data = PRF(master_key, b"client finished", calculate_sha256(transcript), 12)
client_verify_data.hex()

'a29a655e524ba88e143664e2'

In [49]:
encrypted_dtls_record_spec = """
8@content_type-16@tls_version-16@epoch-6B@sequence_number-16@encrypted_data()
"""

In [50]:
from cryptography.hazmat.primitives.ciphers.aead import AESGCM

aad_spec = """16@epoch-6B@sequence_number-8@content_type-16@tls_version-16@data_length"""

def decrypt_packet(packet, key, iv):
    encrypted = packet['encrypted_data']
    if not isinstance(encrypted, bytes):
        encrypted = encrypted.bytes()

    aad = packet.copy()
    aad['data_length'] = len(encrypted) - 16 - 8
    aad_encoded = bytearray(public_construct(aad_spec, aad))
    aad_encoded[:8] = encrypted[:8]

    cipher = AESGCM(key)
    all_server_cert_packet_encoded = cipher.decrypt(iv + encrypted[:8], encrypted[8:], aad_encoded)
    if packet['content_type'].int() == 0x16:
        all_server_cert_packet, _ = public_match(fragment_spec, all_server_cert_packet_encoded)
        packet['tls_handshake'] = all_server_cert_packet
    else:
        packet['decrypted'] = all_server_cert_packet_encoded
    return packet
    

def encrypt_packet(packet, key, iv):
    if packet['content_type'] == 0x16:
        to_encrypt = public_construct(fragment_spec, packet['tls_handshake'])
    else:
        to_encrypt = packet['decrypted']

    aad = packet.copy()
    aad['data_length'] = len(to_encrypt)
    # aad['sequence_number'] = aad['tls_handshake']['message_sequence']
    aad_encoded = public_construct(aad_spec, aad)
    cipher = AESGCM(key)
    encrypted_app_data = cipher.encrypt(iv + aad_encoded[:8], to_encrypt, aad_encoded)
    packet['encrypted_data'] = aad_encoded[:8] + encrypted_app_data
    packet['data_length'] = len(packet['encrypted_data'])
    return packet

In [51]:
finshed_record = {
    "tls_version": 0xfefd,
    "epoch": 1,
    "sequence_number": 0,
    "tls_handshake": {
        'type_and_length': 0x1400000c,
      'message_sequence': 4,
      'fragment_offset': 0x000000,
      'fragment': bytes(client_verify_data)
    },
    "content_type": 22,
    "_selector": 22,
}
encrypted_finished_record = encrypt_packet(finshed_record, client_write_key, client_write_iv)
print(encrypted_finished_record)
encrypted_dtls_record_encoded = public_construct(encrypted_dtls_record_spec, encrypted_finished_record)

# transcript += dtls_record_encoded[13:]
dtls_message_encoded += encrypted_dtls_record_encoded

{'tls_version': 65277, 'epoch': 1, 'sequence_number': 0, 'tls_handshake': {'type_and_length': 335544332, 'message_sequence': 4, 'fragment_offset': 0, 'fragment': b'\xa2\x9ae^RK\xa8\x8e\x146d\xe2'}, 'content_type': 22, '_selector': 22, 'encrypted_data': b'\x00\x01\x00\x00\x00\x00\x00\x00\n\xfeBG\x92\x81{\xeb\x8eo\x1c\xb4\x9f\xf7\xf5h5\xe25Ma\xf7\x86\x88\x0b\x03n\x89\xf0\xa907\xf9\xb8\xb4\xfb[\xd9\xf5\xac', 'data_length': 48}


In [52]:
aaa = public_construct(dtls_record_spec, finshed_record)
print(aaa[13:])
transcript += aaa[13:]
# transcript = transcript[:-len(aaa)+13]

b'\x14\x00\x00\x0c\x00\x04\x00\x00\x00\x00\x00\x0c\xa2\x9ae^RK\xa8\x8e\x146d\xe2'


In [53]:
sock.send(dtls_message_encoded)
dtls_message_encoded = b''

In [72]:
msg = dts_queue.get()
msg

b'\x16\xfe\xfd\x00\x00\x00\x00\x00\x00\x00#\x01\xe2\x04\x00\x01\xd6\x00\x05\x00\x00\x00\x00\x01\xd6\x00\x00\x1c\x03\x01\xd0|\xd0\x9e\xd2\xf6a\xcc\x95\x08\xdc\x85\xbeA\xb1\xa4\x9e\x87\xd9\xed\xd8\xe3\xfe0\xca\xbd\x04>\x06\xa9C\x0b\xc9\xe5"\xe1\x82\x8a\xa4\xee\xacP_\xe7\xe6\xfa\xb8\n#\x99\x8a\xa4\xa6\x13\x13\xda\xca\xa9\x17;\xf9L\x1a\xeb\x99:\x90\xfe\x9cC\xb71\xc0\x98#\xb3svG\xd2\xf9Ho\xbf\x0cno\x82\xe39>0\x97%\x17\x1e\xfdHD\xca\xf1+$LN\\e\xe9\xe3\x9fL\xa6\xba8p~\xd4\x8an\xe8\xd6\x850)3E\'\xd4\xee\xe1\'\xd5\x14\x8f\x03\xffb+P\xf8\xf4*\xe6$]`\x00\xf1\xd0\xb7\x80\xb3\'(\xf6UJ\xe6\x1b\x1e\x18\x9e\xf1ZQq\xf1\xb3\x99\xd1\x88\xd9\xdf}\x16fG,\xa7]-=tV9b[\xc3\xfb\x8c\xe7Qo\xdb\xc8\xcb:\x9bA#\x86\x04R\x89\x04\x00\xf1\xd8\xc9\xde\x1ea\xbb\t\xa9\xb1\xbf\x84\xbcXC\xbf\xf2\xf6\x04\x98\xc8o\xe3\xa56\x99\x04\xfb\xcf\xd7\x1fD\xfbf\xdc|\x95\x08\xff\x19\xf2\xecT\xb7\r\xea\x8cw\x06X$[\xb6\x9c\xfe\xba\r\x9c\xa1\xd4j3\xe8\xaf\x8fL\xff\x1d\xb5\xc2\xd95\xf1\xc8;,g\x88\x07;\x01\x05U\x81\x91\x94\xc8\xa2\xab\xaa\

In [73]:
packet, rem = public_match(dtls_record_spec, msg)
record = packet['tls_handshake']['type_and_length'].bytes() + packet['tls_handshake']['fragment'].bytes()
peer_cert, _ = public_match(handshake_spec, record)
peer_cert

{'new_session_ticket': {'lifetime_hint': 0x00001c03,
  'session_ticket': 0x7cd09ed2f661cc9508dc85be41b1a49e87d9edd8e3fe30cabd043e06a9430bc9e522e1828aa4eeac505fe7e6fab80a23998aa4a61313dacaa9173bf94c1aeb993a90fe9c43b731c09823b3737647d2f9486fbf0c6e6f82e3393e309725171efd4844caf12b244c4e5c65e9e39f4ca6ba38707ed48a6ee8d6853029334527d4eee127d5148f03ff622b50f8f42ae6245d6000f1d0b780b32728f6554ae61b1e189ef15a5171f1b399d188d9df7d1666472ca75d2d3d745639625bc3fb8ce7516fdbc8cb3a9b4123860452890400f1d8c9de1e61bb09a9b1bf84bc5843bff2f60498c86fe3a5369904fbcfd71f44fb66dc7c9508ff19f2ec54b70dea8c770658245bb69cfeba0d9ca1d46a33e8af8f4cff1db5c2d935f1c83b2c6788073b010555819194c8a2abaa154f467bc00a7bf26b2b66416a118b077a52f9b3934bd095fde7db70b453873b439d39d4ca2a762257973ee07697e087f2f4afefcc1e568dff8809aba21710d1db564681faded5737307fbd2eee4fbf6b0d5fcce888f186565bd892b0d93f9779d63a4bd2d6e39e3a60d20946e519e84ef50bd3c0de7d187182ab5bb60a1d98324f16479f43f0ac7c475d545c26290dfbed1425fec78e1cef4cf1e09d94cbe25885be5f1dc34d40

In [74]:
if 'new_session_ticket' not in peer_cert:
    raise RuntimeError("repeated package, retry from dts_queue.get()")
transcript += msg[13:-len(rem)]


In [75]:
packet, rem1 = public_match(dtls_record_spec, rem)
packet

{'tls_version': 0xfefd,
 '_selector': 0x14,
 'epoch': 0x0000,
 'sequence_number': 0x000000000024,
 'change_cipher_spec_message': 0x01}

In [76]:
packet, rem2 = public_match(encrypted_dtls_record_spec, rem1)
packet

{'content_type': 0x16,
 'tls_version': 0xfefd,
 'epoch': 0x0001,
 'sequence_number': 0x000000000000,
 'encrypted_data': 0x000100000000000026e9a92be7e9445d5cb0d0847dacd5bdbcb3439e539e319d40b3733435a59c36af2340ec2d24b1f4}

In [77]:
finished_fragment = decrypt_packet(
    packet,
    server_write_key,
    server_write_iv,
)
finished_fragment

{'content_type': 0x16,
 'tls_version': 0xfefd,
 'epoch': 0x0001,
 'sequence_number': 0x000000000000,
 'encrypted_data': 0x000100000000000026e9a92be7e9445d5cb0d0847dacd5bdbcb3439e539e319d40b3733435a59c36af2340ec2d24b1f4,
 'tls_handshake': {'type_and_length': 0x1400000c,
  'message_sequence': 0x0006,
  'fragment_offset': 0x000000,
  'fragment': 0xa25c5d59c0f731ee253f5014}}

In [78]:
server_verify_data = PRF(master_key, b"server finished", calculate_sha256(transcript), 12)
server_verify_data.hex()

'a25c5d59c0f731ee253f5014'

In [79]:
assert finished_fragment['tls_handshake']['fragment'].bytes() == server_verify_data, "server verify data mismatch"

In [80]:
import crc

In [81]:
sctp_chunks_spec = """
8==1?8@chunk_flags
    -16@chunk_length
    -32@initiate_tag
    -32@a_rwnc
    -16@num_out_streams-16@num_in_streams
    -32@initial_tsn
    -@optional_attributes_encoded
:8==2?8@chunk_flags
    -16@chunk_length
    -32@initiate_tag
    -32@a_rwnc
    -16@num_out_streams-16@num_in_streams
    -32@initial_tsn
    -@optional_attributes_encoded
:8==10?8@chunk_flags
    -16@chunk_length
    -@cookie
:8==11?8@chunk_flags
    -16@chunk_length
:8==0?4@res
    -1@i-1@u-1@b-1@e
    -16@chunk_length
    -32@tsn
    -16@s-16@n
    -32@payload_protocol_id
    -@data
:8==3?8@flags
    -16@chunk_length
    -32@tsn_ack
    -32@a_rwnd
    -16@n_gap_ack_blocks
    -16@n_duplicate_tsns
    -@gap_ack_duplicate_tsns
:8==4?8@flags
    -16@chunk_length
    -@optional_attributes_encoded
:8==5?8@flags
    -16@chunk_length
    -@optional_attributes_encoded
"""

optional_header_spec = """16@type-16@length"""
sctp_chunk_header_spec = "8@chunk_type-8@chunk_flags-16@chunk_size"

def parse_sctp_chunk(sctp_chunk):
    sctp_chunk_header, _ = public_match(sctp_chunk_header_spec, sctp_chunk)
    data_len = sctp_chunk_header['chunk_size'].int()
    sctp_chunk_value, sctp_chunk = sctp_chunk[:data_len], sctp_chunk[math.ceil(data_len /4) * 4:]
    res, _ = public_match(sctp_chunks_spec, sctp_chunk_value)
    if 'optional_attributes_encoded' in res:
        oares = res['optional_attributes_encoded'].bytes()
        res['optional_attributes'] = []
        while oares:
            optional_attribute_header, _ = public_match(optional_header_spec, oares)
            optional_attribute_header['data'] = oares[4:optional_attribute_header['length'].int()]
            oares = oares[optional_attribute_header['length'].int():]
            res['optional_attributes'].append(optional_attribute_header)
    return res, sctp_chunk
            
        

In [82]:
sctp_common_header_spec = """
16@source_port
-16@dst_port
-32@verification_tag
-32@checksum"""

def get_sctp_message(dtls_message):
    packet, _ = public_match(encrypted_dtls_record_spec, dtls_message)
    decrypted = decrypt_packet(
        packet,
        server_write_key,
        server_write_iv,
    )
    raw = decrypted['decrypted']
    raw_clone = bytearray(raw)
    raw_clone[8:12] = b'\0\0\0\0'
    crc_res = public_construct('32@data', {'data': crc.crc(raw_clone)})[::-1]
    
    sctp_header, rem = public_match(sctp_common_header_spec, raw)
    assert crc_res == sctp_header['checksum'].bytes(), "invalid checksum"
    chunks = []
    while rem:
        chunk, rem = parse_sctp_chunk(rem)
        chunks.append(chunk)
    return {"header": sctp_header, "chunks": chunks, "epoch": decrypted["epoch"], "sequence_number": decrypted["sequence_number"]}

In [83]:
optional_full_header_spec = """16@type-16@length-@data"""
def construct_sctp_chunk(sctp_chunk):
    if 'optional_attributes' in sctp_chunk:
        sctp_chunk['optional_attributes_encoded'] = b''
        for attr in sctp_chunk['optional_attributes']:
            attr['length'] = len(attr['data']) + 4
            sctp_chunk['optional_attributes_encoded'] += public_construct(optional_full_header_spec, attr)
    sctp_chunk['chunk_length'] = 0
    res = public_construct(sctp_chunks_spec, sctp_chunk)
    sctp_chunk['chunk_length'] = len(res)
    res = public_construct(sctp_chunks_spec, sctp_chunk)
    if sctp_chunk['chunk_length'] % 4 != 0:
        res += b'\0' * (4-(sctp_chunk['chunk_length']%4))
    return res
    

In [84]:
def create_sctp_message(sctp_message):
    sctp_message['header']['checksum'] = 0
    buff = public_construct(sctp_common_header_spec, sctp_message['header'])
    for chunk in sctp_message['chunks']:
        buff += construct_sctp_chunk(chunk)

    buff = bytearray(buff)
    crc_res = public_construct('32@data', {'data': crc.crc(buff)})[::-1]
    buff[8:12] = crc_res
    
    packet = {
        "tls_version": 0xfefd,
        "epoch": sctp_message["epoch"],
        "sequence_number": sctp_message["sequence_number"],
        "decrypted": bytes(buff),
        "content_type": 23,
        "_selector": 23,
    }
    encrypted = encrypt_packet(
        packet,
        client_write_key,
        client_write_iv,
    )
    return public_construct(encrypted_dtls_record_spec, encrypted)
    

In [85]:
while True:
    msg = dts_queue.get()
    if msg[0] != 0x17:
        print('not an app data, retrying')
        continue
    print('got app data')
    break

sctp_init = get_sctp_message(msg)
client_verification_tag = sctp_init['chunks'][0]['initiate_tag']
sctp_init

got app data


{'header': {'source_port': 0x1388,
  'dst_port': 0x1388,
  'verification_tag': 0x00000000,
  'checksum': 0x2fe11126},
 'chunks': [{'chunk_flags': 0x00,
   '_selector': 0x01,
   'chunk_length': 0x001e,
   'initiate_tag': 0x83674e37,
   'a_rwnc': 0x00500000,
   'num_out_streams': 0xffff,
   'num_in_streams': 0xffff,
   'initial_tsn': 0x8778db80,
   'optional_attributes_encoded': 0xc00000048008000682c0,
   'optional_attributes': [{'type': 0xc000, 'length': 0x0004, 'data': b''},
    {'type': 0x8008, 'length': 0x0006, 'data': b'\x82\xc0'}]}],
 'epoch': 0x0001,
 'sequence_number': 0x000000000001}

In [86]:
tsn = 0xe306eef2
own_dtls_seq = 2

In [87]:
# sctp_init = create_sctp_message({'header': {'source_port': 0x1388,
#   'dst_port': 0x1388,
#   'verification_tag': 0x00000000,
#   'checksum': 0x9e2630c2},
#  'chunks': [{'chunk_flags': 0x00,
#    '_selector': 0x01,
#    'chunk_length': 0x001e,
#    'initiate_tag': 0x6ee08c30,
#    'a_rwnc': 0x00500000,
#    'num_out_streams': 0xffff,
#    'num_in_streams': 0xffff,
#    'initial_tsn': tsn,
#    'optional_attributes_encoded': 0xc00000048008000682c0,
#    'optional_attributes': [{'type': 0xc000, 'length': 0x0004, 'data': b''},
#     {'type': 0x8008, 'length': 0x0006, 'data': b'\x82\xc0'}]}],
#  'epoch': 0x0001,
#  'sequence_number': 0x000000000007})
# sock.send(sctp_init)

In [88]:
sctp_init_ack = create_sctp_message({'header': {'source_port': 0x1388,
  'dst_port': 0x1388,
  'verification_tag': client_verification_tag,
  'checksum': 0x378def6c},
 'chunks': [{'chunk_flags': 0x00,
   '_selector': 0x02,
   'chunk_length': 0x0052,
   'initiate_tag': 0xff448058,
   'a_rwnc': 0x00500000,
   'num_out_streams': 0xffff,
   'num_in_streams': 0xffff,
   'initial_tsn': tsn,
   'optional_attributes_encoded': 0x0007003164635343545030306ee08c30ff4480589170927fe306eef200500000000000000000000001000100ffffffff00000000c00000048008000682c0,
   'optional_attributes': [{'type': 0x0007,
     'length': 0x0031,
     'data': b'dcSCTP00n\xe0\x8c0\xffD\x80X\x91p\x92\x7f\xe3\x06\xee\xf2\x00P\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x01\x00\xff\xff\xff\xff\x00'},
    {'type': 0x0000,
     'length': 0x00c0,
     'data': b'\x00\x00\x04\x80\x08\x00\x06\x82\xc0'}]}],
 'epoch': 0x0001,
 'sequence_number': own_dtls_seq})
own_dtls_seq += 1
sock.send(sctp_init_ack)


In [89]:
# while True:
#     msg = dts_queue.get()
#     pkt = get_sctp_message(msg)
#     if pkt['chunks'][0]['_selector'].int() != 2:
#         continue
#     print('found init ack')    
#     cookie = next(i['data'] for i in pkt['chunks'][0]['optional_attributes'] if i['type'].int() == 7)
#     server_verification_tag = pkt['header']['verification_tag']
#     break
# pkt

In [90]:
while True:
    msg = dts_queue.get()
    pkt = get_sctp_message(msg)
    if pkt['chunks'][0]['_selector'].int() != 10:
        print('not found cookie echo, got', pkt['chunks'][0]['_selector'].int(), "instead")    
        continue
    print('found cookie echo')
    break
pkt

not found cookie echo, got 1 instead
not found cookie echo, got 1 instead
not found cookie echo, got 1 instead
not found cookie echo, got 1 instead
not found cookie echo, got 1 instead
found cookie echo


{'header': {'source_port': 0x1388,
  'dst_port': 0x1388,
  'verification_tag': 0xff448058,
  'checksum': 0xa804049c},
 'chunks': [{'chunk_flags': 0x00,
   '_selector': "
   ",
   'chunk_length': 0x0031,
   'cookie': 0x64635343545030306ee08c30ff4480589170927fe306eef200500000000000000000000001000100ffffffff00}],
 'epoch': 0x0001,
 'sequence_number': 0x000000000007}

In [91]:
# sctp_cookie_echo = create_sctp_message({'header': {'source_port': 0x1388,
#   'dst_port': 0x1388,
#   'verification_tag': server_verification_tag},
#  'chunks': [{'chunk_flags': 0x00,
#    '_selector': 10,
#     'cookie': cookie}],
#  'epoch': 0x0001,
#  'sequence_number': 0x000000000009})
# sock.send(sctp_cookie_echo)

In [92]:
sctp_cookie_ack = create_sctp_message({'header': {'source_port': 0x1388,
  'dst_port': 0x1388,
  'verification_tag': client_verification_tag},
 'chunks': [{'chunk_flags': 0x00,
   '_selector': 11}],
 'epoch': 0x0001,
 'sequence_number': own_dtls_seq})
own_dtls_seq += 1
sock.send(sctp_cookie_ack)

In [93]:
# sctp_cookie_ack = create_sctp_message({'header': {'source_port': 0x1388,
#   'dst_port': 0x1388,
#   'verification_tag': client_verification_tag},
#  'chunks': [{'chunk_flags': 0x00,
#    '_selector': 11}],
#  'epoch': 0x0001,
#  'sequence_number': 0x000000000009})
# sock.send(sctp_cookie_ack)

In [94]:
sctp_data_sequence_number = 0

In [95]:
data_send = create_sctp_message({'header': {'source_port': 0x1388,
  'dst_port': 0x1388,
  'verification_tag': client_verification_tag,
  'checksum': 0x14fad29b},
 'chunks': [{'res': 0b0000,
   '_selector': 0x00,
   'i': 0b0,
   'u': 0b0,
   'b': 0b1,
   'e': 0b1,
   'chunk_length': 0x0013,
   'tsn': tsn,
   's': 0x0000,
   'n': sctp_data_sequence_number,
   'payload_protocol_id': 0x00000033,
   'data': "hello, my name is mie2"}],
 'epoch': 0x0001,
 'sequence_number': own_dtls_seq})
own_dtls_seq += 1
tsn += 1
sctp_data_sequence_number += 1
sock.send(data_send)

In [99]:
msg = dts_queue.get()
data_ack_received = get_sctp_message(msg)
data_ack_received

{'header': {'source_port': 0x1388,
  'dst_port': 0x1388,
  'verification_tag': 0xff448058,
  'checksum': 0xe02faf3e},
 'chunks': [{'flags': 0x00,
   '_selector': 0x03,
   'chunk_length': 0x0010,
   'tsn_ack': 0xe306eef2,
   'a_rwnd': 0x00480000,
   'n_gap_ack_blocks': 0x0000,
   'n_duplicate_tsns': 0x0000,
   'gap_ack_duplicate_tsns': ''}],
 'epoch': 0x0001,
 'sequence_number': 0x00000000000b}

In [101]:
msg = dts_queue.get()
data_received = get_sctp_message(msg)
data_received

{'header': {'source_port': 0x1388,
  'dst_port': 0x1388,
  'verification_tag': 0xff448058,
  'checksum': 0x65f59797},
 'chunks': [{'res': 0b0000,
   '_selector': 0x00,
   'i': 0b0,
   'u': 0b0,
   'b': 0b1,
   'e': 0b1,
   'chunk_length': 0x0013,
   'tsn': 0x8778db80,
   's': 0x0000,
   'n': 0x0000,
   'payload_protocol_id': 0x00000033,
   'data': "hey"}],
 'epoch': 0x0001,
 'sequence_number': 0x00000000000d}

In [102]:
sack = create_sctp_message({'header': {'source_port': 0x1388,
  'dst_port': 0x1388,
  'verification_tag': client_verification_tag,
  'checksum': 0xe02faf3e},
 'chunks': [{'flags': 0x00,
   '_selector': 0x03,
   'chunk_length': 0x0010,
   'tsn_ack': data_received['chunks'][-1]['tsn'],
   'a_rwnd': 0x00480000,
   'n_gap_ack_blocks': 0x0000,
   'n_duplicate_tsns': 0x0000,
   'gap_ack_duplicate_tsns': ''}],
 'epoch': 0x0001,
 'sequence_number': own_dtls_seq})
own_dtls_seq += 1
sock.send(sack)

In [110]:
msg = dts_queue.get()
heartbeat_received = get_sctp_message(msg)
heartbeat_received

{'header': {'source_port': 0x1388,
  'dst_port': 0x1388,
  'verification_tag': 0xff448058,
  'checksum': 0xe2771ad8},
 'chunks': [{'flags': 0x00,
   '_selector': 0x04,
   'chunk_length': 0x0010,
   'optional_attributes_encoded': 0x0001000c000000000510b789,
   'optional_attributes': [{'type': 0x0001,
     'length': 0x000c,
     'data': b'\x00\x00\x00\x00\x05\x10\xb7\x89'}]}],
 'epoch': 0x0001,
 'sequence_number': 0x000000000015}

In [111]:
heartbeat_ack = create_sctp_message({'header': {'source_port': 0x1388,
  'dst_port': 0x1388,
  'verification_tag': client_verification_tag,
  'checksum': 0x30743be0},
 'chunks': [{'flags': 0x00,
   '_selector': 0x05,
   'chunk_length': 0x0010,
   'optional_attributes_encoded': 0x0001000c00000000a92c0797,
   'optional_attributes': [{'type': 0x0001,
     'length': 0x000c,
     'data': heartbeat_received['chunks'][0]['optional_attributes'][0]['data']}]}],
 'epoch': 0x0001,
 'sequence_number': own_dtls_seq})
own_dtls_seq += 1
sock.send(heartbeat_ack)

In [112]:
background_sctp_handler_running = False
try:
    background_sctp_thread.join()
except:
    pass
def message_handler(msg):
    global own_dtls_seq
    decoded = get_sctp_message(msg)
    ret = []
    for chunk in decoded['chunks']:
        if chunk['_selector'].int() == 0: # data
            sack = create_sctp_message({'header': {'source_port': 0x1388,
                'dst_port': 0x1388,
                'verification_tag': client_verification_tag,
                'checksum': 0xe02faf3e},
                'chunks': [{'flags': 0x00,
                    '_selector': 0x03,
                    'chunk_length': 0x0010,
                    'tsn_ack': chunk['tsn'],
                    'a_rwnd': 0x00480000,
                    'n_gap_ack_blocks': 0x0000,
                    'n_duplicate_tsns': 0x0000,
                    'gap_ack_duplicate_tsns': ''}],
                'epoch': 0x0001,
                'sequence_number': own_dtls_seq})
            own_dtls_seq += 1
            sock.send(sack)
            ret.append(chunk['data'])
        elif chunk['_selector'].int() == 4: # heartbeat
            heartbeat_ack = create_sctp_message({'header': {'source_port': 0x1388,
                'dst_port': 0x1388,
                'verification_tag': client_verification_tag,
                'checksum': 0x30743be0},
                'chunks': [{'flags': 0x00,
                    '_selector': 0x05,
                    'chunk_length': 0x0010,
                    'optional_attributes_encoded': 0x0001000c00000000a92c0797,
                    'optional_attributes': [{'type': 0x0001,
                    'length': 0x000c,
                    'data': chunk['optional_attributes'][0]['data']}]}],
                'epoch': 0x0001,
                'sequence_number': own_dtls_seq})
            own_dtls_seq += 1
            sock.send(heartbeat_ack)
    return ret


In [113]:
background_sctp_handler_running = True
def background_sctp_handler():
    while background_sctp_handler_running:
        msg = dts_queue.get()
        print(msg)
        print(message_handler(msg))
    print("exiting")

background_sctp_thread = threading.Thread(target=background_sctp_handler)
background_sctp_thread.start()

b'\x17\xfe\xfd\x00\x01\x00\x00\x00\x00\x00\x16\x004\x00\x01\x00\x00\x00\x00\x00\x16L\xb4\xfe\xee\x91\ni3\xd6\xa6\xf9xe\x87\x1b\x15`6)\xc8\xc7\xbe\xf9\x17\x18\x93)\x04\xa3\tE\xd2]9\xba\x85\xb8v\xce\xe8\xa67\xcd\\'
[]
b'\x17\xfe\xfd\x00\x01\x00\x00\x00\x00\x00\x17\x008\x00\x01\x00\x00\x00\x00\x00\x17\xe2H\x16\x16\x9f3p\x1d\xed\xd9_\xbd\xf8\xa6\xb8+\xacs=\x8c\xe6\xa5\x9ej\xba\x82+\xaeF\x84kA\xba@\xa3G\xa5\x9a\x83\x14\x1d\xa7<)\xd8\xc5\xab\xdc'
["hi"]
exiting


In [114]:
def send_message(data):
    global own_dtls_seq
    global tsn
    global sctp_data_sequence_number
    data_send = create_sctp_message({'header': {'source_port': 0x1388,
        'dst_port': 0x1388,
        'verification_tag': client_verification_tag,
        'checksum': 0x14fad29b},
        'chunks': [{'res': 0b0000,
            '_selector': 0x00,
            'i': 0b0,
            'u': 0b0,
            'b': 0b1,
            'e': 0b1,
            'chunk_length': 0x0013,
            'tsn': tsn,
            's': 0x0000,
            'n': sctp_data_sequence_number,
            'payload_protocol_id': 0x00000033,
            'data': data}],
        'epoch': 0x0001,
        'sequence_number': own_dtls_seq})
    own_dtls_seq += 1
    tsn += 1
    sctp_data_sequence_number += 1
    sock.send(data_send)

In [115]:
send_message("hello a second time")

In [116]:
background_sctp_handler_running = False
background_sctp_thread.join()

In [117]:
background_sctp_handler_running = True
def background_sctp_handler():
    while background_sctp_handler_running:
        msg = dts_queue.get()
        ret = message_handler(msg)
        for i in ret:
            print('>', i.str())
    print("exiting")

background_sctp_thread = threading.Thread(target=background_sctp_handler)
background_sctp_thread.start()
while True:
    send_message(input())

 hello 3


> hi 4


 asd


> asd


KeyboardInterrupt: Interrupted by user